## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
#!pip install gmaps
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key


# Configure gmaps API key
gmaps.configure(api_key = g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City,Cloudiness,Country,Date,Humidity,Temp,Lat,Lng,Wind Speed,Current Discription
0,flinders,57,AU,1626376210,77,57.06,-34.5833,150.8552,5.01,broken clouds
1,mahebourg,18,MU,1626376211,74,71.89,-20.4081,57.7000,14.65,light rain
2,tuy hoa,100,VN,1626376211,91,76.08,13.0833,109.3000,6.40,overcast clouds
3,jamestown,20,US,1626375939,62,84.18,42.0970,-79.2353,10.00,few clouds
4,sorland,100,NO,1626376212,88,51.87,67.6670,12.6934,42.77,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = int(input("What is the minimum temperature in degrees you would like for the location Degree Celcius: "))


What is the minimum temperature in degrees you would like for the location Degree Celcius: 50


In [4]:
max_temp = int(input("What is the maximum temperature in degrees you would like for the location Degree Celcius: "))

What is the maximum temperature in degrees you would like for the location Degree Celcius: 80


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
df1 = city_data_df.loc[city_data_df["Temp"]<max_temp,:]
df2 = df1.loc[df1["Temp"]>min_temp,:]
df2.head()

,City,Cloudiness,Country,Date,Humidity,Temp,Lat,Lng,Wind Speed,Current Discription
0,flinders,57,AU,1626376210,77,57.06,-34.5833,150.8552,5.01,broken clouds
1,mahebourg,18,MU,1626376211,74,71.89,-20.4081,57.7000,14.65,light rain
2,tuy hoa,100,VN,1626376211,91,76.08,13.0833,109.3000,6.40,overcast clouds
4,sorland,100,NO,1626376212,88,51.87,67.6670,12.6934,42.77,light rain
5,rikitea,5,PF,1626376212,73,73.11,-23.1203,-134.9692,9.37,clear sky


In [6]:
# 4a. Determine if there are any empty rows.

df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 820 entries, 0 to 1371
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 820 non-null    object 
 1   Cloudiness           820 non-null    int64  
 2   Country              810 non-null    object 
 3   Date                 820 non-null    int64  
 4   Humidity             820 non-null    int64  
 5   Temp                 820 non-null    float64
 6   Lat                  820 non-null    float64
 7   Lng                  820 non-null    float64
 8   Wind Speed           820 non-null    float64
 9   Current Discription  820 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 70.5+ KB


In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
df3 = df2.dropna(axis = 0, how = "any")
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 810 entries, 0 to 1371
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 810 non-null    object 
 1   Cloudiness           810 non-null    int64  
 2   Country              810 non-null    object 
 3   Date                 810 non-null    int64  
 4   Humidity             810 non-null    int64  
 5   Temp                 810 non-null    float64
 6   Lat                  810 non-null    float64
 7   Lng                  810 non-null    float64
 8   Wind Speed           810 non-null    float64
 9   Current Discription  810 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 69.6+ KB


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = df3[["City", "Country", "Temp", "Current Discription", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Temp,Current Discription,Lat,Lng,Hotel Name
0,flinders,AU,57.06,broken clouds,-34.5833,150.8552,
1,mahebourg,MU,71.89,light rain,-20.4081,57.7000,
2,tuy hoa,VN,76.08,overcast clouds,13.0833,109.3000,
4,sorland,NO,51.87,light rain,67.6670,12.6934,
5,rikitea,PF,73.11,clear sky,-23.1203,-134.9692,
6,port alfred,ZA,56.14,clear sky,-33.5906,26.8910,
8,hermanus,ZA,55.90,clear sky,-34.4187,19.2345,
9,busselton,AU,53.10,overcast clouds,-33.6500,115.3333,
10,yellowknife,CA,50.56,light rain,62.4560,-114.3525,
12,los llanos de aridane,ES,73.13,broken clouds,28.6585,-17.9182,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
   

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

       
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        
    
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    #print(hotels)
      
    #6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except:
                      
        pass
    
        

In [10]:
hotel_df.head()

,City,Country,Temp,Current Discription,Lat,Lng,Hotel Name
0,flinders,AU,57.06,broken clouds,-34.5833,150.8552,Shellharbour Resort & Conference Centre
1,mahebourg,MU,71.89,light rain,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
2,tuy hoa,VN,76.08,overcast clouds,13.0833,109.3000,CenDeluxe Hotel Tuy Hoa
4,sorland,NO,51.87,light rain,67.6670,12.6934,Sjybrygga fiskecamp
5,rikitea,PF,73.11,clear sky,-23.1203,-134.9692,People ThankYou


In [11]:
# 7. Drop the rows where there is no Hotel Name.
print(hotel_df.isnull().sum())
clean_hotel_df = hotel_df.dropna(axis = 0, how = "any")


City                   0
Country                0
Temp                   0
Current Discription    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64


In [12]:
# 8a. Create the output File (CSV)
clean_hotel_df.head()

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv('WeatherPy_vacation', index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
!jupyter nbextension enable --py gmaps
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [15]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))